## implement img2col via tf.image.extract_image_patches in TensorFlow  

> see this stackoverflow question [Implementing im2col in TensorFlow](https://stackoverflow.com/questions/45553929/implementing-im2col-in-tensorflow)  

In [1]:
# !/usr/bin/env python3
# -*-coding:utf-8-*-
# @file: epe_loss_test.py
# @brief:
# @author: Changjiang Cai, ccai1@stevens.edu, caicj5351@gmail.com
# @version: 0.0.1
# @creation date: 12-01-2019
# @last modified: Thu 28 Feb 2019 12:11:23 AM EST
import tensorflow as tf
import numpy as np

""" Implementing im2col in TensorFlow """
# > see https://stackoverflow.com/questions/45553929/implementing-im2col-in-tensorflow;
if __name__ == '__main__':
    channels = 2
    image = np.arange(10 * 10 * channels).reshape(1, 10, 10, channels)
    images = tf.convert_to_tensor(image.astype(np.float32))

    filter_size = 3
    sobel_x = tf.constant([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], tf.float32)
    # a filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    sobel_x_filter = tf.reshape(sobel_x, [3, 3, 1, 1])
    sobel_x_filter = tf.tile(sobel_x_filter, [1,1,channels,1])

    image_patches = tf.extract_image_patches(
            images,[1, filter_size, filter_size, 1],
            [1, 1, 1, 1], [1, 1, 1, 1],
            padding='SAME')

    #actual = tf.reduce_sum(tf.multiply(image_patches, tf.reshape(sobel_x_filter, [9])), 3, keep_dims=True)
    actual = tf.reduce_sum(tf.multiply(image_patches, tf.reshape(sobel_x_filter, [filter_size*filter_size*channels])), 3, keepdims=True)
    expected = tf.nn.conv2d(images, sobel_x_filter, strides=[1, 1, 1, 1], padding='SAME')
    res = tf.reduce_sum(expected - actual)

    with tf.Session() as sess:
            print "err = ", sess.run(res)
            print "image_patches shape = ", sess.run(tf.shape(image_patches))
            print "input image[0, 0:3,0:3,:]  = ", image[0,0:3,0:3,:]
            print "im2col image_patche[0,1,1,:] = ", sess.run(image_patches)[0,1,1,:]

/home/ccj/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


err =  0.0
image_patches shape =  [ 1 10 10 18]
input image[0, 0:3,0:3,:]  =  [[[ 0  1]
  [ 2  3]
  [ 4  5]]

 [[20 21]
  [22 23]
  [24 25]]

 [[40 41]
  [42 43]
  [44 45]]]
im2col image_patche[0,1,1,:] =  [ 0.  1.  2.  3.  4.  5. 20. 21. 22. 23. 24. 25. 40. 41. 42. 43. 44. 45.]


The first $3 \times 3$ patch is:  
![im2col-test](./files/im2col-test.png)

The im2col operation will generate:  
`image_patche[0,1,1,:] =  [ 0.  1.  2.  3.  4.  5. 20. 21. 22. 23. 24. 25. 40. 41. 42. 43. 44. 45.]`
That is `(channels, channels, channels, ..., channels)`